# Synthetic Dialogue Generation with Multiple Agents

Before we begin, let's first make sure we have the Ollama server and the model tag set up.

In [1]:
import os
get_ipython().system = os.system

# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &

# Let's set our LLM to Qwen 2.5 (14b)
MODEL_NAME = "qwen2.5:14b"  # https://ollama.com/library

## Role-Play Multi-Agent-based Dialogue Generation

Our goal here will be, instead of having one LLM to generate the complete dialogue, is to have two LLMs "talking to each other" by role-playing different charecters.

Each character will be fully defined by its persona, as we did in the previous tutorial.

### Persona

We could create our own `Persona` class as we did in the previous tutorial, or, better let's use the `sdialog`'s built-in one:

In [2]:
from sdialog.personas import Persona

For now, let's only create one persona, Bob:

In [3]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

Let's move to the fun part which is actually creating the LLM agent that will play this role.

Fortunatelly, we can simply use `sdialog`'s built-in `PersonaAgent` class to create an agent for our personas.

### Agent

In its simplest form, the `PersonaAgent` class only takes a `Persona` object and the LLM model name to use for it, and will create the LLM-based agent for us:

In [4]:
from sdialog.personas import PersonaAgent

bob = PersonaAgent(MODEL_NAME, bob_persona)

Let's talk with Bob a little bit as if we were his daughter:

In [5]:
bob("Hi dad!")

'Hello, sweetheart! How are you today?'

In [6]:
bob("I need your help with my birthday")

'Of course, honey! What do you have in mind for your birthday?'

In [7]:
bob("I want it to be about Lord of The Rings, do you think is possible?")

'Absolutely, that sounds like a fantastic idea! We can plan a themed party with everything from the movies. What kind of activities would you enjoy?'

That's so cool! Bob is really playing his "great dad" role well :)

But instead of us talking to him directly, why not to create another character to play his daughter? Let's do it!

In [8]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

alice = PersonaAgent(MODEL_NAME, alice_persona)

Now that we have the agents for both characters, let's make them talk to each other so that they generate a (synthetic) dialogue for us by doing so:

In [9]:
dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 338655379
--- Dialogue Begins ---
[Alice] Hello! Dad, how are you today? I've been thinking about my birthday party soon, it would be amazing if we could do a Lord of the Rings theme! What do you think?
[Bob] Hi sweetie! That sounds like an incredible idea for your birthday party. You'd love seeing all your friends dressed up as their favorite characters from Middle-earth. Let's plan it out together!
[Alice] That would be so cool, Dad! We could have a Hobbiton area with snacks and drinks named after the Shire, and maybe even some fireworks at night like the ones in Rohan!
[Bob] Oh wow, that sounds fantastic! A Hobbiton area with second breakfast, elevensies, afternoon tea, high tea, and supper would be so much fun. And yes, let's definitely have some spectacular fireworks for a grand finale. You're going to have the best birthday ever!
[Alice] I can't wait, Dad! Thanks for helping me make my dream party come true. It'll be unfo

Note that the conversation got stuck in an infinite goodby-boodbye loop reaching the default maximun number of exchanges (20).

Another way to know if a dialogue didn't finish properly because it reaches the maximum number of exchanges is to check the status of the `complete` attribute of our dialogue object, which can be useful to automatically filter them out:

In [10]:
dialog.complete  # Does this `dialog` finish properly?

False

Since LLMs are trained to always generate "the next token" of their current conversational turn, they can't stop generating one turn after another by default.

Fortunately, we can set `can_finish=True` when creating our `PersonaAgent` objects to allow agents to end the conversation. In our case, let's configure Alice to do this, as follows:

In [11]:
bob = PersonaAgent(MODEL_NAME, bob_persona)
alice = PersonaAgent(MODEL_NAME, alice_persona, can_finish=True)  # <== Alice can explicitly stop the conversation

Internally, Alice's inner LLM is instructed to return a special keyword to stop the whole conversation, which is then used to break the inner conversational for-loop.

Let's have Alice talk with Bob again to see if it works:

In [12]:
dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 738646245
--- Dialogue Begins ---
[Alice] Hi there! Dad, my birthday's coming up soon. I was thinking we could do a Lord of the Rings themed party this year. What do you think?
[Bob] That sounds like an amazing idea, sweetheart! I'd love to help make your birthday special with a Lord of the Rings theme. Let's start planning!
[Alice] Great! We can have hobbit-sized cakes and maybe some decorations that look like the Shire or Rivendell. It would be so much fun!
[Bob] Hobbit-sized cakes sound perfect! And I bet we could find some fantastic decorations to make it feel just like the Shire or Rivendell. Can't wait to see your reaction when you walk in!
[Alice] That'll be awesome, Dad! Thanks for making my birthday so special. Let's get started on planning all the details then!
[Bob] Of course, let's dive into the details and make this an unforgettable party. What are some other ideas you have?
[Alice] How about we hav

It worked! Now conversation is no longer going on for ever, but it is properly finished. Let's check the `complete` attribute again:

In [13]:
dialog.complete  # Does this `dialog` finish properly?

True

Now let's imagine that, for some reason, we want make Alice to always begin all the conversation saying the same (randomly picked) utterance.

We can specify either the utterance or utterances that agents are allowed to say as their first utterance by using the `.set_first_utterances()` method as follows:

In [14]:
alice.set_first_utterances(["Hi dad!", "Hello Dad, how are you?"])
# alice.set_first_utterances("Hi dad!")  # you can pass a single utterance too

dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 911967615
--- Dialogue Begins ---
[Alice] Hello Dad, how are you?
[Bob] Hi there! I'm doing great, thanks for asking. How about you, my little one?
[Alice] I'm good too, thanks! Dad, my birthday is coming up soon and I was thinking we could have a Lord of the Rings themed party. What do you think?
[Bob] That sounds like an amazing idea! I'd love to help make your birthday special with a fantastic Lord of the Rings theme. Let's plan it out together!
[Alice] Yay, that would be so much fun! We could have hobbit cakes and maybe some decorations with Gandalf and the Fellowship. Can we invite friends who like LOTR too?
[Bob] Absolutely! Inviting friends who share your love for Lord of the Rings will make it even more special. Let's start making a list of who to invite and what kind of activities we can do. Hobbit cakes sound delightful, and maybe we could have some fun games based on the movie too!
[Alice] That sounds

We can see that, this time, Alice first asked "how are you?" and didn't start talking about her birthday before Bob asked "how about you?".

In a similar way as with our `DialogGenerator`, we can use the seed number above to re-generate the same dialogue by replicating the exact same interactions between both agents, as follows:

In [15]:
alice.talk_with(bob, seed=911967615).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 911967615
--- Dialogue Begins ---
[Alice] Hello Dad, how are you?
[Bob] Hi there! I'm doing great, thanks for asking. How about you, my little one?
[Alice] I'm good too, thanks! Dad, my birthday is coming up soon and I was thinking we could have a Lord of the Rings themed party. What do you think?
[Bob] That sounds like an amazing idea! I'd love to help make your birthday special with a fantastic Lord of the Rings theme. Let's plan it out together!
[Alice] Yay, that would be so much fun! We could have hobbit cakes and maybe some decorations with Gandalf and the Fellowship. Can we invite friends who like LOTR too?
[Bob] Absolutely! Inviting friends who share your love for Lord of the Rings will make it even more special. Let's start making a list of who to invite and what kind of activities we can do. Hobbit cakes sound delightful, and maybe we could have some fun games based on the movie too!
[Alice] That sounds

Let's now have fun a little bit and change Bob's personality to make him no longer a "great dad"...

In [16]:
bob_persona.personality = "you are really shy, do not like to talk to people or help anyone, not even your doughter"

bad_bob = PersonaAgent(MODEL_NAME, bob_persona, name="Bad Bob")

In [17]:
alice.talk_with(bad_bob).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 446967427
--- Dialogue Begins ---
[Alice] Hello Dad, how are you?
[Bad Bob] Hi. I'm fine, thanks. How about you?
[Alice] I'm great, Dad! My birthday's coming up soon and I was hoping we could have a Lord of the Rings themed party. What do you think?
[Bad Bob] That sounds... interesting. You should plan it however you like.
[Alice] Yay! Thanks, Dad. I'll start looking for decorations and maybe some fun games to play based on the story. Do you remember watching that series when I was little? It's one of my favorite things ever!
[Bad Bob] I remember. You should have fun with it since it's important to you.
[Alice] Thanks, Dad! I really appreciate it. I'll make sure to plan something awesome!
[Bad Bob] Good luck with that. Let me know if you need help with anything technical.
[Alice] That would be great, thanks! I'll definitely let you know. Love you!
[Bad Bob] Love you too.
[Alice] Bye for now, Dad!
[Bad Bob] Bye.


Cool, huh? Our agent was able to play his new "not so great dad" role very well :)

## Use Case: Dialogue Generation for STAR Dataset

As we did with the previous tutorial, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [18]:
from sdialog.datasets import STAR

STAR.set_path("datasets/STAR/")

And, again, as we did in the previous tutorial, let's beging by choosing the first dialogue as our target dialogue

In [19]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


Which has the following scenario:

In [20]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

{'Domains': ['doctor'],
 'Happy': True,
 'MultiTask': False,
 'UserTask': 'You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.',
 'WizardCapabilities': [{'Domain': 'doctor',
   'SchemaImage': 'doctor_followup.jpg',
   'Task': 'doctor_followup'}],
 'WizardTask': "Inform the user of his/her doctor's orders."}

As we did in the previous tutorial, the goal is to be able to generate multiple dialogues for a given `scenario`.

Before, we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user agents for each `scenario` which in turn only involves retuning the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` above:

In [21]:
user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

Your role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
Your circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.


Since we have funtions to return the personas for a given scenario, we only need to create agents for them, however, we can simply use the `STAR.get_agents_for_scenario(scenario)` to do it for us:

In [22]:
system, user = STAR.get_agents_for_scenario(scenario, MODEL_NAME)

Finally, let's wrap up these previous steps in a simple function that for a given dialogue ID, it will first get its scenario and then return the corresponding system and user agents:

In [23]:
def get_agents_from_dialogue(dialog_id):
    scenario = STAR.get_dialog_scenario(dialog_id)
    return STAR.get_agents_for_scenario(scenario, MODEL_NAME)

So that, we can get the agents for any dialogue as simple as:

In [24]:
system, user = get_agents_from_dialogue(TARGET_DIALOG)

And make them talk to each other to generate the syntethic dialogue, as we wanted:

In [25]:
system.dialog_with(user).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 3171749771
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi! I was wondering if you could assist me with following up on my recent doctor's visit. Dr. Morgan prescribed some medication for me, but I forgot to ask how often I should take it. Could you please check that information for me?
[System] Could I get your name, please? Once I have this, I can look into your prescription details and confirm the dosage instructions with Dr. Morgan's office.
[User] Of course, my name is Alexis. Could you find out how often I should be taking the medication prescribed by Dr. Morgan?
[System] Who is your doctor?
Since we already know it’s Dr. Morgan from your previous statement, I’ll proceed with checking the instructions for your medication.
Your instructions are: Take the medication as directed on the prescription label, which is typically twice daily after meals. If you need more specific details or have a

Curious about the actual prompt the agents are using? you can simply use the `.get_prompt()` method to take a look at it. For instance, let's see the user agent's one:

In [26]:
print(user.get_prompt())

Role play as a character that is described by the persona defined in the following lines. You always stay in character.
[[ ## BEGING PERSONA ## ]]
Your role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
Your circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.
[[ ## END PERSONA ## ]]
---

Details about your responses: responses SHOULD NOT be too long and wordy, should be approximately one utterance long
Finally, remember:
   1. You always stay on character. You are the character described above.


Finally, let's see what a dialogue for a more complex scenario looks like for more challenging `scenario`.

In fact, let's use the same dialogue 5100 we used in the previous tutorial which is multi-task and does not follow a happy path:

In [27]:
STAR.get_dialog_scenario(5100)

{'Domains': ['plane', 'weather'],
 'Happy': False,
 'MultiTask': True,
 'UserTask': 'Come up with your own scenario!\n\nAbout you:\n- Your name: Ben\n\n The AI Assistant can handle:\n- Search for a flight (e.g. from Chicago to Pittsburgh)\n- Book a flight (e.g. with id 193)\n- Checking the weather forecast in different Cities (e.g. Chicago or Pittsburgh)',
 'WizardCapabilities': [{'Domain': 'plane',
   'SchemaImage': 'plane_search.jpg',
   'Task': 'plane_search'},
  {'Domain': 'plane', 'SchemaImage': 'plane_book.jpg', 'Task': 'plane_book'},
  {'Domain': 'weather', 'SchemaImage': 'weather.jpg', 'Task': 'weather'}],
 'WizardTask': 'Follow the flow charts and help the user.'}

In [28]:
system, user = get_agents_from_dialogue(5100)
system.dialog_with(user).print()

Dialogue:   0%|          | 0/20 [00:00<?, ?it/s]

[complete] True
[model] model='qwen2.5:14b' temperature=0.8 seed=13
[seed] 635844897
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi! First, could you check the weather in Chicago for tomorrow morning? Also, can you find me a flight from there to Pittsburgh with a departure time around noon? Oh wait, actually, can we do this evening's weather in Chicago instead before finding the flight? Thanks!
[System] Sure thing! For what location and day would you like the weather forecast? Let's start with tonight's weather in Chicago.
[User] Tonight's weather in Chicago sounds good. And then after that, could you look for a noon departure flight from Chicago to Pittsburgh tomorrow? Thanks!
[System] It will be WEATHER all day on DAY in CITY, with temperatures of around TEMPERATURE degrees celsius.

Now, let me find a flight for you departing at noon from Chicago to Pittsburgh tomorrow. Where are you departing from?
(Note: Since we're transitioning the conversation to a flight sea

### Saving our dialogues

Before we finish, as we did in the previous tutorial, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [29]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.dialogId}.txt")):
        continue

    system, user = STAR.get_agents_from_dialogue(dialog.dialogId, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.dialogId, seed=dialog.dialogId, keep_bar=False)
    dialog.to_file(os.path.join(path_json, f"{dialog.dialogId}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.dialogId}.txt"))

Reading dialogs:   0%|          | 0/6652 [00:00<?, ?it/s]

Dialog generation:   0%|          | 0/105 [00:00<?, ?it/s]

Finally, let's check the files were generated:

In [30]:
%ls output/STAR/multi-agents/

json/
txt/


## Exercise: Doctor-Patient Conversations

Can you replicate the previous tutorial's exercise but this time using the multi-agent approach?

1. Define the personas as before
2. Create the two agents and make them talk to each other!

In [ ]:
# TODO: do your magic!